# 6.4. Чисельне розв'язування крайових задач для звичайних диференціальних рівнянь
---------------------

## 6.4.3.  Розв'язування крайових задач для ЗДР методом скінчених різниць (сіток) 

Розглянемо крайову задачу 

$(1)\quad\qquad\qquad\qquad u''+q(x)u=f(x), \quad x \in (0,l),$

$(2)\quad\qquad\qquad\qquad \alpha_0u'(0)+\beta_0u(0)=\gamma_0,$

$(3)\quad\qquad\qquad\qquad\alpha_1u'(l)+\beta_1u(l)=\gamma_1,$

де $q, f\in C([0,l])$, а $\alpha_0,  \beta_0, \gamma_0, \alpha_1,  \beta_1, \gamma_1$ -- сталі, причому  
$|\alpha_{0}|+|\beta_{0}|> 0,\quad |\alpha_{1}|+|\beta_{1}|> 0$.

Нехай ця задача має і тільки один розв'язок $C^2[0,l]$.  

Побудуємо сітку з $N+1, \; N\in \mathbb{N},$ точок
$x_i=ih,\quad i=\overline{0,N}$, де $h:=\dfrac{l}{N}$.

У методі скінчених різниць чисельним розв'язком крайової задачі (1)-(3) є наближення сіткових значень $u_i,\, i=\overline{0,N},$  розв'язку   $u$ у вузлах $x_i,\, i=\overline{0,N}.$

Після заміни похідних першого і другого порядку від функції $u$ у вузлах сітки різницевими співвідношеннями матимемо СЛАР 

$(4)\quad\qquad\qquad\qquad \sum_{i=0}^{N}c_{ij}u_i=d_j,\quad j=\overline{0,N},$

з квадратною тридіагональною матрицею
$$
C=
\begin{pmatrix}
-\alpha_0+\beta_0h& \alpha_0& 0& \ldots& 0& 0& 0\\
1& h^2q_1-2& 1& \ldots& 0& 0& 0\\
0& 1& h^2q_2-2& \ldots& 0& 0& 0\\
\ldots& \ldots& \ldots& \ldots& \ldots& \ldots& \ldots\\
0& 0& 0& \ldots& 1& h^2q_{N-1}-2& 1\\
0& 0& 0& \ldots& 0& -\alpha_1& \alpha_1+h\beta_1
\end{pmatrix}
$$
і вектором вільних членів $d$, де 
$d_0 = h\gamma_0, \; d_N= h\gamma_1, \; d_i=h^2 f(x_i),\; i=\overline{1,N-1}$.

Беручи до уваги діагональну структуру матриці, розв'язок СЛАР (4) знаходимо методом прогонки. Отриманий вектор і буде шуканим чисельним розв'язком крайової задачі (1)-(3). 

#### Пояснення до використання програмного коду
-----------------
*   Підготувати потрібні функції : 
    1.   виконати комірку для підготовки середовища 
    2.   виконати комірки, де **визначені** функції ``FDA_solver``,``set_matrix_diagonals_and_vector`` і ``tridiagonal_matrix_algorithm``    

*    Обчислити чисельний розв'язок конкретної крайової задачі :
     1.  виконати комірку, де **визначені** функції ``q``  і ``f``, які задають конкретну крайову задачу 
     2.  виконати комірку, в якій задано усі параметри крайової задачі 
     3.  виконати комірку з викликом функції ``FDA_solver``, перед виконанням задати відповідні аргументи цієї функції.

*    Задання параметра сітки і обчислення матриці та вектора вільних членів СЛАР :
     1.   виконати комірку, в якій задано параметр  $N$
     2.   виконати комірку з викликом функції set_matrix_diagonals_and_vector

*   Знайти чисельний розв'язок крайової задачі:
     1. Виконати комірку з викликом функції tridiagonal_matrix_algorithm

#### Програмна реалізація методів
------------

>#### Підготовка середовища

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

>#### ``FDA_solver`` -- функція, яка отримує вхідні параметри чисельного методу, формує масиви значень розв'язку на межі області, задає матрицю і вектор вільних членів  СЛАР, знаходить значення чисельного розв'язку у внутрішніх вузлах сітки і повертає вектор, який містить значення розв'язку у всіх точках сітки. За замовчуванням ( при ``plotting=True``) будуються графіки чисельного розв'язку задачі.
    


In [2]:
def FDA_solver(f,q, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N):
    """ Розв'язування крайових задач (1)-(3) 
        методом скінчених різниць 
    """      
    c,a,b,d,x = set_matrix_diagonals_and_vector(f,q,alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)
    u = tridiagonal_matrix_algorithm(a,b,c,d)
    return u,x

>### ``set_matrix_diagonals_and_vector`` -- функція для задання діагоналей матриці $C$ і вектора $d$

In [3]:
def set_matrix_diagonals_and_vector(f,q,alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N):
    """ функція задає 3 діагоналі матриці і вектор вільних членів СЛАР """
    
    h=l/N
    h2=h*h
    x=np.linspace(0, l, N+1)
    
    c = np.empty(N+1, dtype=float ) 
    for i in range(1,N): 
        c[i] = h2*q(x[i])-2
    c[0] = -alpha0+h*beta0
    c[N] =  alpha1+h*beta1   
    
    a=np.ones(N+1, dtype=float)
    a[0] = 0
    a[N] = -alpha1   
    
    b=np.ones(N+1, dtype=float)
    b[0] = alpha0
    b[N] = 0 
    
    d = np.empty(N+1, dtype=float ) 
    for i in range(1,N): 
        d[i] = h2*f(x[i])
    d[0]=h*gamma0
    d[N]=h*gamma1
    
    return c,a,b,d,x

>#### ``tridiagonal_matrix_algorithm`` -- функція, яка реалізує метод прогонки для розв'язування СЛАР

In [4]:
def tridiagonal_matrix_algorithm(a,b,c,g):
    """ метод прогонки для розв'язування СЛАР
        з 3-діагональною матрицею 
        вектор с-головна діагональ
        вектори a i b - нижня і верхня діагоналі, паралельні головній
        вектор g - вільні члени
      """
    n1=c.size
   
    alpha = np.empty(n1, dtype=float ) 
    beta  = np.empty(n1, dtype=float )
    
    if c[0] !=0 :
        alpha[0] =-b[0]/c[0]
        beta [0] = g[0]/c[0]
    else:
        raise Exception('c[0]==0') 
    
    for i in range(1,n1):
        w=a[i]*alpha[i-1]+c[i]
        if w != 0 :
            alpha[i] =-b[i]/w
            beta[i]  = (g[i] - a[i]*beta[i-1])/w
        else:
            raise Exception('w==0')
        
    x = np.empty(n1, dtype=float )
    n = n1-1
    x[n] = beta[n]
    for i in range(n-1,-1,-1):
        x[i] = alpha[i]*x[i+1] + beta[i]
    return x

>#### ``f`` і ``q`` -- функції, які задають конкретне рівняння 

#### Обчислювальні експерименти
------------

Продемонструємо застосування методу скінчених різниць для знаходження чисельного розв'язку задачі (1)-(3).

##### Приклад 1.
Отримати методом скінчених різниць чисельний розв'язок крайової задачі 

$ u'' + u =1 $ на відрізку $[0, \pi/2]$, $\; u(0)=0, \; u'(\pi/2)=1$. 

Легко бачити, що функція $u(x) = 1 - cos(x)$ є розв'язком такої задачі.

Отож, задамо відомі функції, що фігурують в рівнянні:

In [50]:
def f1(x):
    return 1
def q1(x):
    return 1

А також значення усіх параметрів задачі:

In [51]:
l=np.pi/2

alpha0=0
beta0=1
gamma0=0

alpha1=1
beta1=0
gamma1=1

Для фіксування початкового розміру сітки будемо використовувати параметр ``N_start``. Задамо його значення і спочатку обчислимо на досить густій сітці значення аналітичного (точного) розв'язку, а потім знайдемо чисельні розв'язки задачі на послідовності сіток, подвоюючи кожного разу кількість вузлів:

In [52]:
N_start=5
x=np.linspace(0, l, N_start*64+1)
u = 1 - np.cos(x)

In [53]:
N = N_start
u_0,x0 = FDA_solver(f1,q1, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)

N *= 2 
u_1,x1 = FDA_solver(f1,q1, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)

N *= 2 
u_2,x2 = FDA_solver(f1,q1, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)

N *= 2 
u_3,x3 = FDA_solver(f1,q1, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)

N *= 2 
u_4,x4 = FDA_solver(f1,q1, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)

N *= 2 
u_5,x5 = FDA_solver(f1,q1, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)

Побудуємо графіки отриманих  розв'язків:

In [54]:
fig = plt.figure(figsize=(8, 5))
plt.plot(x, u, label='u')
plt.plot(x0, u_0, label='u_0')
plt.plot(x1, u_1, label='u_1')

plt.plot(x2, u_2, label='u_2')
plt.plot(x3, u_3, label='u_3')
plt.plot(x4, u_4, label='u_4')
plt.plot(x5, u_5, label='u_5')
ax = fig.gca()
ax.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Зазначимо, що бібліотечна функція ``plot`` при побудові графіків виконує лінійну інтерполяцію по значеннях, заданих у вузлах сітки. Нагадаємо також, що в режимі ``Zoom to rectangle`` можна маштабувати зображення на графічній панелі.

Отож, як видно з отриманих графіків, чисельні розв'язки швидко збігаються до точного розв'язку. 

Для точнішого аналізу збережемо отримані дані в таблиці, розглядаючи чисельні розв'язки лише на множині вузлів, яка є перетином побудованої послідовності сіток:

In [32]:
df=pd.DataFrame({'x':x0[1:x0.size-1:],'u_0':u_0[1:u_0.size-1:],'u_1':u_1[2:u_1.size-2:2],'u_2':u_2[4:u_2.size-4:4],'u_3':u_3[8:u_3.size-8:8],'u_4':u_4[16:u_4.size-16:16],'u_5':u_5[32:u_5.size-32:32],'u':u[64:u.size-64:64]})
df

,x,u_0,u_1,u_2,u_3,u_4,u_5,u
0,0.314159,0.072848,0.060959,0.054977,0.051968,0.050458,0.049701,0.048943
1,0.628319,0.237203,0.214025,0.202507,0.196749,0.193867,0.192425,0.190983
2,0.942478,0.476843,0.444186,0.428141,0.420167,0.416188,0.414201,0.412215
3,1.256637,0.768116,0.728865,0.709781,0.700350,0.695659,0.693319,0.690983


Обчислимо абсолютні похибки отриманих розв'язків:

In [33]:
df1=pd.DataFrame()
df1['e_0']=np.abs(df['u']-df['u_0'])
df1['e_1']=np.abs(df['u']-df['u_1'])
df1['e_2']=np.abs(df['u']-df['u_2'])
df1['e_3']=np.abs(df['u']-df['u_3'])
df1['e_4']=np.abs(df['u']-df['u_4'])
df1['e_5']=np.abs(df['u']-df['u_5'])
df1

,e_0,e_1,e_2,e_3,e_4,e_5
0,0.023905,0.012015,0.006034,0.003025,0.001515,0.000758
1,0.046220,0.023042,0.011524,0.005766,0.002884,0.001442
2,0.064628,0.031971,0.015927,0.007952,0.003973,0.001986
3,0.077133,0.037882,0.018798,0.009367,0.004676,0.002336


Побудуємо графіки похибок в логарифмічній шкалі:

In [34]:
fig = plt.figure(figsize=(8, 5))
df1.e_0.plot(logy=True)
df1.e_1.plot(logy=True)
df1.e_2.plot(logy=True)
df1.e_3.plot(logy=True)
df1.e_4.plot(logy=True)
df1.e_5.plot(logy=True)
ax = fig.gca()
ax.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Обчислимо для кожного чисельного розв'язку його абсолютну похибку ``ne_*`` за нормою $\| \cdot \|_\infty$:

In [35]:
ne_0 = max(np.abs(df1['e_0']))
ne_1 = max(np.abs(df1['e_1']))
ne_2 = max(np.abs(df1['e_2']))
ne_3 = max(np.abs(df1['e_3']))
ne_4 = max(np.abs(df1['e_4']))
ne_5 = max(np.abs(df1['e_5']))
print(f" ne_0={ne_0:1.2e}\n ne_1={ne_1:1.2e}\n ne_2={ne_2:1.2e}\n ne_3={ne_3:1.2e}\n ne_4={ne_4:1.2e}\n ne_5={ne_5:1.2e}")

 ne_0=7.71e-02
 ne_1=3.79e-02
 ne_2=1.88e-02
 ne_3=9.37e-03
 ne_4=4.68e-03
 ne_5=2.34e-03


Як бачимо, подвоєння кількості вузлів сітки зумовлює зменшення вдвічі зазначеної похибки. Якщо порахуємо так звану швидкість збіжності, то отримаємо дуже близьке до 2 значення:

In [37]:
df2=pd.DataFrame()
df2['r_0'] = df1['e_0'] / df1['e_1']
df2['r_1'] = df1['e_1'] / df1['e_2']
df2['r_2'] = df1['e_2'] / df1['e_3']
df2['r_3'] = df1['e_3'] / df1['e_4']
df2['r_4'] = df1['e_4'] / df1['e_5']

df2

,r_0,r_1,r_2,r_3,r_4
0,1.989555,1.991292,1.994750,1.997147,1.998516
1,2.005872,1.999455,1.998844,1.999198,1.999543
2,2.021432,2.007431,2.002890,2.001237,2.000566
3,2.036130,2.015192,2.006883,2.003263,2.001587


##### Приклад 2.
(Приклад 6.5) Отримати методом скінчених різниць чисельний розв'язок крайової задачі 

$ u'' - (x+1)u = x^2 $ на відрізку $[0, 3]$, $\; u(0)=0, \; u(3)=0$. 

Виконуємо ті самі кроки, що і в попередньому прикладі. Відмінність лише у тому, що точний розв'язок такої задачі невідомий.

Отож, визначимо потрібні функції та задамо значення параметрів задачі:

In [39]:
def f2(x):
    return x*x

def q2(x):
    return -(x+1)

l=3

alpha0=0
beta0=1
gamma0=0

alpha1=0
beta1=1
gamma1=0

Знаходимо чисельні розв'язки на послідовності сіток, у яких на кожному кроці подвоюється кількість вузлів:

In [40]:
N_start=10
N = N_start
u_0,x0 = FDA_solver(f2,q2, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)
N *= 2 
u_1,x1 = FDA_solver(f2,q2, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)
N *= 2 
u_2,x2 = FDA_solver(f2,q2, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)
N *= 2 
u_3,x3 = FDA_solver(f2,q2, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)
N *= 2 
u_4,x4 = FDA_solver(f2,q2, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)
N *= 2 
u_5,x5 = FDA_solver(f2,q2, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)
N *= 2 
u_6,x6 = FDA_solver(f2,q2, alpha0,alpha1,beta0,beta1, gamma0, gamma1,l,N)

Побудуємо графіки отриманих розв'язків:

In [41]:
fig = plt.figure(figsize=(8, 5))

plt.plot(x0, u_0, label='u_0')
plt.plot(x1, u_1, label='u_1')
plt.plot(x2, u_2, label='u_2')
plt.plot(x3, u_3, label='u_3')
plt.plot(x4, u_4, label='u_4')
plt.plot(x5, u_5, label='u_5')
plt.plot(x6, u_6, label='u_6')
ax = fig.gca()
ax.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Візуально спостерігаємо збіжність чисельних розв'язків до деякої функції. Для кількісного аналізу збережемо дані в таблиці:

In [42]:
df=pd.DataFrame({'x':x0[1:x0.size-1:],'u_0':u_0[1:u_0.size-1:],'u_1':u_1[2:u_1.size-2:2],'u_2':u_2[4:u_2.size-4:4],'u_3':u_3[8:u_3.size-8:8],'u_4':u_4[16:u_4.size-16:16],'u_5':u_5[32:u_5.size-32:32],'u_6':u_6[64:u_6.size-64:64]})
df

,x,u_0,u_1,u_2,u_3,u_4,u_5,u_6
0,0.3,-0.143329,-0.144571,-0.144885,-0.144964,-0.144983,-0.144988,-0.144989
1,0.6,-0.295327,-0.297509,-0.298062,-0.298200,-0.298235,-0.298243,-0.298245
2,0.9,-0.457452,-0.460504,-0.461278,-0.461472,-0.461521,-0.461533,-0.461536
3,1.2,-0.624901,-0.628958,-0.629989,-0.630248,-0.630312,-0.630329,-0.630333
4,1.5,-0.786481,-0.791835,-0.793200,-0.793543,-0.793629,-0.793651,-0.793656
5,1.8,-0.922520,-0.929538,-0.931334,-0.931786,-0.931899,-0.931928,-0.931935
6,2.1,-0.999433,-1.008353,-1.010646,-1.011223,-1.011368,-1.011404,-1.011413
7,2.4,-0.958288,-0.968669,-0.971350,-0.972026,-0.972196,-0.972238,-0.972249
8,2.7,-0.691980,-0.701283,-0.703699,-0.704309,-0.704462,-0.704500,-0.704509


Оскільки точний розв'язок невідомий, то дослідимо, як швидко чисельні розв'язки збігаються до ``u_6``:

In [43]:
df1=pd.DataFrame()
df1['e_0']=np.abs(df['u_6']-df['u_0'])
df1['e_1']=np.abs(df['u_6']-df['u_1'])
df1['e_2']=np.abs(df['u_6']-df['u_2'])
df1['e_3']=np.abs(df['u_6']-df['u_3'])
df1['e_4']=np.abs(df['u_6']-df['u_4'])
df1['e_5']=np.abs(df['u_6']-df['u_5'])
df1

,e_0,e_1,e_2,e_3,e_4,e_5
0,0.001661,0.000418,0.000104,0.000026,0.000006,0.000001
1,0.002919,0.000736,0.000184,0.000045,0.000011,0.000002
2,0.004084,0.001032,0.000258,0.000064,0.000015,0.000003
3,0.005431,0.001375,0.000344,0.000085,0.000020,0.000004
4,0.007174,0.001821,0.000456,0.000113,0.000027,0.000005
5,0.009415,0.002397,0.000600,0.000149,0.000035,0.000007
6,0.011980,0.003060,0.000767,0.000190,0.000045,0.000009
7,0.013960,0.003580,0.000898,0.000222,0.000053,0.000011
8,0.012530,0.003226,0.000811,0.000201,0.000048,0.000010


Графіки (у логарифмічній шкалі) демонструють однаковий порядок отриманих похибок на усьому відрізку ``l``:

In [44]:
fig = plt.figure(figsize=(8, 5))
df1.e_0.plot(logy=True)
df1.e_1.plot(logy=True)
df1.e_2.plot(logy=True)
df1.e_3.plot(logy=True)
df1.e_4.plot(logy=True)
df1.e_5.plot(logy=True)
ax = fig.gca()
ax.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Обчислимо для кожного чисельного розв'язку його абсолютну похибку за нормою $\| \cdot \|_\infty$:

In [45]:
ne_0 = max(np.abs(df1['e_0']))
ne_1 = max(np.abs(df1['e_1']))
ne_2 = max(np.abs(df1['e_2']))
ne_3 = max(np.abs(df1['e_3']))
ne_4 = max(np.abs(df1['e_4']))
ne_5 = max(np.abs(df1['e_5']))
print(f" ne_0={ne_0:1.2e}\n ne_1={ne_1:1.2e}\n ne_2={ne_2:1.2e}\n ne_3={ne_3:1.2e}\n ne_4={ne_4:1.2e}\n ne_5={ne_5:1.2e}")

 ne_0=1.40e-02
 ne_1=3.58e-03
 ne_2=8.98e-04
 ne_3=2.22e-04
 ne_4=5.30e-05
 ne_5=1.06e-05


Якщо обчислити швидкість збіжності (до ``u_6``), то матимемо число, близьке до 4, яке має тенденцію до збільшення при збільшенні вузлів сітки:

In [46]:
df2=pd.DataFrame()
df2['r_0'] = df1['e_0'] / df1['e_1']
df2['r_1'] = df1['e_1'] / df1['e_2']
df2['r_2'] = df1['e_2'] / df1['e_3']
df2['r_3'] = df1['e_3'] / df1['e_4']
df2['r_4'] = df1['e_4'] / df1['e_5']

df2

,r_0,r_1,r_2,r_3,r_4
0,3.973235,4.004190,4.045699,4.199501,4.999852
1,3.965843,4.002246,4.045203,4.199372,4.999813
2,3.958273,4.000242,4.044690,4.199239,4.999773
3,3.949734,3.997981,4.044112,4.199088,4.999729
4,3.939694,3.995326,4.043433,4.198911,4.999676
5,3.927997,3.992234,4.042642,4.198706,4.999615
6,3.914706,3.988711,4.041740,4.198471,4.999545
7,3.899928,3.984778,4.040733,4.198208,4.999466
8,3.883733,3.980444,4.039621,4.197919,4.999380


In [47]:
plt.close('all')